In [1]:
import os
import wandb
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from PIL import Image

plt.style.use('seaborn')
%matplotlib inline

### Image Size

In [2]:
img = Image.open('dataset/train/smile/file0001.jpg')
img.size

(64, 64)

In [3]:
for img in os.listdir('dataset/train/smile'):
    img = Image.open(f'dataset/train/smile/{img}')
    assert img.size == (64, 64)

In [4]:
for img in os.listdir('dataset/train/not_smile'):
    img = Image.open(f'dataset/train/not_smile/{img}')
    assert img.size == (64, 64)

In [5]:
for img in os.listdir('dataset/test/smile'):
    img = Image.open(f'dataset/test/smile/{img}')
    assert img.size == (64, 64)

In [6]:
for img in os.listdir('dataset/test/not_smile'):
    img = Image.open(f'dataset/test/not_smile/{img}')
    assert img.size == (64, 64)

### Hyperparameters

In [7]:
config = {
    'seed': 42,
    'img_size': 64,
    'color_mode': 'rgb',
    'batch_size': 64,
    'epochs': 50,
    'lr': 1e-3,
}

### Dataset

In [8]:
print("\nTraining Dataset")
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory='dataset/train',
    validation_split=0.2,
    subset='training',
    label_mode='binary',
    color_mode=config['color_mode'],
    seed=config['seed'],
    image_size=(config['img_size'], config['img_size']),
    batch_size=config['batch_size'],
)

print("\nValidation Dataset")
val_ds = tf.keras.utils.image_dataset_from_directory(
    directory='dataset/train',
    validation_split=0.2,
    subset='validation',
    label_mode='binary',
    color_mode=config['color_mode'],
    seed=config['seed'],
    image_size=(config['img_size'], config['img_size']),
    batch_size=config['batch_size'],
    shuffle=False
)

print("\nTesting Dataset")
test_ds = tf.keras.utils.image_dataset_from_directory(
    directory='dataset/test',
    label_mode='binary',
    color_mode=config['color_mode'],
    seed=config['seed'],
    image_size=(config['img_size'], config['img_size']),
    batch_size=config['batch_size'],
    shuffle=False
)


Training Dataset
Found 2800 files belonging to 2 classes.
Using 2240 files for training.
Metal device set to: Apple M1

Validation Dataset
Found 2800 files belonging to 2 classes.
Using 560 files for validation.

Testing Dataset
Found 1200 files belonging to 2 classes.


2022-06-07 11:44:25.618072: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-07 11:44:25.618195: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
train_ds.class_names

['not_smile', 'smile']

In [9]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

### Model Building

In [11]:
if config['color_mode'] == 'rgb':
    input_shape = (config['img_size'], config['img_size'], 3)
else:
    input_shape = (config['img_size'], config['img_size'], 1)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Rescaling(1./255, input_shape=input_shape))

model.add(tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=config['lr']),
              metrics='binary_accuracy')

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        1

### Training

In [12]:
wandb.init(config=config, project='Smile-Detector', name='baseline2')

wandb: Currently logged in as: gautamj. Use `wandb login --relogin` to force relogin


In [13]:
mc = tf.keras.callbacks.ModelCheckpoint(
    filepath='models/weights-{epoch:03d}-{val_loss:.4f}.hdf5',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
)

wb = wandb.keras.WandbCallback()

callbacks = [mc, wb]

history = model.fit(train_ds, validation_data=val_ds, epochs=config['epochs'],
                    callbacks=callbacks, verbose=2)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
2022-06-07 11:44:34.488699: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/50


2022-06-07 11:44:34.845140: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


35/35 - 2s - loss: 0.6457 - binary_accuracy: 0.6281 - val_loss: 0.4763 - val_binary_accuracy: 0.8821 - _timestamp: 1654582476.0000 - _runtime: 9.0000 - 2s/epoch - 47ms/step


2022-06-07 11:44:35.938391: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 2/50
35/35 - 1s - loss: 0.5249 - binary_accuracy: 0.7415 - val_loss: 0.5082 - val_binary_accuracy: 0.7536 - _timestamp: 1654582476.0000 - _runtime: 9.0000 - 783ms/epoch - 22ms/step
Epoch 3/50
35/35 - 1s - loss: 0.4080 - binary_accuracy: 0.8161 - val_loss: 0.2029 - val_binary_accuracy: 0.9250 - _timestamp: 1654582477.0000 - _runtime: 10.0000 - 834ms/epoch - 24ms/step
Epoch 4/50
35/35 - 1s - loss: 0.3366 - binary_accuracy: 0.8562 - val_loss: 0.1899 - val_binary_accuracy: 0.9321 - _timestamp: 1654582478.0000 - _runtime: 11.0000 - 804ms/epoch - 23ms/step
Epoch 5/50
35/35 - 1s - loss: 0.2860 - binary_accuracy: 0.8808 - val_loss: 0.2051 - val_binary_accuracy: 0.9054 - _timestamp: 1654582479.0000 - _runtime: 12.0000 - 753ms/epoch - 22ms/step
Epoch 6/50
35/35 - 1s - loss: 0.2546 - binary_accuracy: 0.8897 - val_loss: 0.3009 - val_binary_accuracy: 0.8536 - _timestamp: 1654582480.0000 - _runtime: 13.0000 - 813ms/epoch - 23ms/step
Epoch 7/50
35/35 - 1s - loss: 0.2263 - binary_accuracy: 0.897

Epoch 46/50
35/35 - 1s - loss: 0.0014 - binary_accuracy: 1.0000 - val_loss: 0.1982 - val_binary_accuracy: 0.9643 - _timestamp: 1654582510.0000 - _runtime: 43.0000 - 737ms/epoch - 21ms/step
Epoch 47/50
35/35 - 1s - loss: 0.0012 - binary_accuracy: 1.0000 - val_loss: 0.1877 - val_binary_accuracy: 0.9661 - _timestamp: 1654582511.0000 - _runtime: 44.0000 - 739ms/epoch - 21ms/step
Epoch 48/50
35/35 - 1s - loss: 0.0010 - binary_accuracy: 1.0000 - val_loss: 0.2053 - val_binary_accuracy: 0.9643 - _timestamp: 1654582512.0000 - _runtime: 45.0000 - 738ms/epoch - 21ms/step
Epoch 49/50
35/35 - 1s - loss: 8.8596e-04 - binary_accuracy: 1.0000 - val_loss: 0.2030 - val_binary_accuracy: 0.9643 - _timestamp: 1654582512.0000 - _runtime: 45.0000 - 734ms/epoch - 21ms/step
Epoch 50/50
35/35 - 1s - loss: 7.8656e-04 - binary_accuracy: 1.0000 - val_loss: 0.1975 - val_binary_accuracy: 0.9661 - _timestamp: 1654582513.0000 - _runtime: 46.0000 - 741ms/epoch - 21ms/step


In [14]:
fig, ax = plt.subplots()
loss = history.history['loss']
val_loss = history.history['val_loss']
ax.plot(loss, label='loss')
ax.plot(val_loss, label='val_loss')
ax.legend()
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
wandb.log({'Loss Chart': fig})
plt.show()

/Users/gautam/miniforge3/lib/python3.9/site-packages/plotly/matplotlylib/renderer.py:611: UserWarning:

I found a path object that I don't think is part of a bar chart. Ignoring.



In [15]:
fig, ax = plt.subplots()
acc = history.history['binary_accuracy']
val_acc = history.history['val_binary_accuracy']
ax.plot(acc, label='acc')
ax.plot(val_acc, label='val_acc')
ax.legend()
ax.set_xlabel('Epochs')
ax.set_ylabel('Accuracy')
wandb.log({'Acc Chart': fig})
plt.show()

### Testing

In [16]:
model = tf.keras.models.load_model('models/baseline.hdf5')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 16, 16, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        1

In [17]:
test_loss, test_acc = model.evaluate(test_ds)
wandb.log({'test_loss': test_loss})
wandb.log({'test_acc': test_acc})

2022-06-07 11:45:14.051443: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 0s 14ms/step - loss: 0.6934 - binary_accuracy: 0.8542


In [18]:
wandb.finish()

binary_accuracy,▁▃▅▅▆▆▆▇▇▇▇▇▇██████████████████▇████████
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▅▅▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁
test_acc,▁
test_loss,▁
val_binary_accuracy,▅▁▆▇▄▅▅▇▇▆▇▇▇▇▇▇███▇▇▇▇█▇▇█▇▇▇▇██▇▇▇████
val_loss,▇█▃▃▅▃▄▂▁▂▂▂▂▁▂▂▂▂▁▂▂▂▄▁▃▃▁▅▅▄▃▁▂▃▃▃▃▃▃▃
best_epoch,29
best_val_loss,0.08107
binary_accuracy,1.0
epoch,49
